In [91]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

def scrape_proyecto_nuevo(soup, r):
    

    info_dict = {}
    info_dict["Localizacion:"] = [soup.find(name="div", attrs={"class":"d3-property-details__detail-label"}).string]
    #info_dict["Categoria:"] = [i.text for i in elementos_tabla if i.text.lower().find("categor")!=-1]
    info_dict["avance_proyecto"] = [soup.find(name="span",attrs={"class":"d3-new-property-stage__current-status--counter"}).text]
    info_dict["estado_proyecto"] = [soup.find(name="p", attrs={"class":"d3-new-property-stage__current-status"}).text]
    info_dict["Detalles"] = [soup.find(name="div", attrs={"class":"d3-property-about d3-property-about--new"}).text]
    info_dict["id"] = [r.url.split("/")[-1]]
    info_dict["link"] = [r.url]
    
    tabla_modelos = soup.find(name="div", attrs={"class":"d3-property-details d3-property-details--new"})
    elementos_tabla = soup.find_all(name="div", attrs={"class":"d3-property-details__detail-label"})

    info_dict["Localizacion:"] = [i.text for i in elementos_tabla if i.text.lower().find("localizaci")!=-1]
    info_dict["Categoria:"] = [i.text for i in elementos_tabla if i.text.lower().find("categor")!=-1]

    df_tmp = pd.DataFrame()

    #for elemento in elementos_tabla:
        

    print(info_dict)
    
    return pd.DataFrame(info_dict)


In [92]:
link="https://www.encuentra24.com/el-salvador-es/bienes-raices-proyectos-nuevos/comalapa-flats/24582258"
r = requests.get(link)
soup = BeautifulSoup(r.text, "html.parser")

df = scrape_proyecto_nuevo(soup,r)
df

{'Localizacion:': ['Localización La Paz\n'], 'avance_proyecto': ['1/4 '], 'estado_proyecto': ['1/4 Preventa'], 'Detalles': ['\nAcerca de\n\nDescubra la esencia del refinamiento y la comodidad en nuestras exclusivas casas de una planta. Un oasis de elegancia contemporánea diseñado para aquellos que aprecian la sofisticación sin complicaciones. Disfrute de espacios luminosos y abiertos que se entrelazan con la naturaleza en armonía. Dé un paso hacia el lujo discreto y una vida sin escaleras. Bienvenido a su nuevo hogar, donde la elegancia y la frescura convergen en perfecta sintonía. \n\n90 Viviendas \n3 Tipologías \n2 Parqueos por vivienda \n\nAmenidades: \n- Piscina \n- Área de niños \n- Zonas verdes \n- 1/2 Cancha de BKB y Football Casa Club \n\nReserva: $5,000 \nPrima: 10%\n\n\nLeer más\n\n'], 'id': ['24582258'], 'link': ['https://www.encuentra24.com/el-salvador-es/bienes-raices-proyectos-nuevos/comalapa-flats/24582258'], 'Categoria:': ['Categoria Proyectos nuevos\n']}


,Localizacion:,avance_proyecto,estado_proyecto,Detalles,id,link,Categoria:
0,Localización La Paz\n,1/4,1/4 Preventa,\nAcerca de\n\nDescubra la esencia del refinam...,24582258,https://www.encuentra24.com/el-salvador-es/bie...,Categoria Proyectos nuevos\n
